In [1]:
import ete_lib
import rst_lib

In [2]:
paths, files = rst_lib.build_file_map()

annotation_pairs = [
    rst_lib.build_annotation_pair(files, paths, identifier)
    for identifier in files[rst_lib.TRAIN][rst_lib.DOUBLE]
]

2
2
2
2
2
2
Did left child,  2

2
2
2
Did left child,  2

Did left child,  2

Did left child,  2

3
2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  3

2
2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
Did left child,  2

2
2
Did left child,  2

Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

2
Did left child,  2

2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

2
2
Did left child,  2

2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
2
2
Did left child,  2

Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
Did 

Did left child,  2

2
2
2
Did left child,  2

2
Did left child,  2

Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
2
2
Did left child,  2

Did left child,  2

Did left child,  2

Did left child,  2

2
Did left child,  2

2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
Did left child,  2

2
Did left child,  2

2
2
Did left child,  2

2
Did left child,  2

Did left child,  2

3
2
Did left child,  2

Did left child,  3

2
2
Did left child,  2

Did left child,  2

Did left child,  2

2
2
2
Did left child,  2

Did left child,  2

2
2
Did left child,  2

Did left child,  2

Did left child,  2

Did left child,  2

2
Did left child,  2

2
Did left child,  2

2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

4
2
2
Did left child,  2

Did left child,  2

Did left child,  4

3
2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  3

2
2
Did left child,  2

2
2
Did left child, 

2
2
2
2
2
2
2
2
2
Did left child,  2

Did left child,  2

Did left child,  2

Did left child,  2

3
2
Did left child,  2

3
2
2
Did left child,  2

Did left child,  2

Did left child,  3

2
2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  3

2
2
2
2
Did left child,  2

Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

Did left child,  2

3
2
2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
2
Did left child,  2

Did left child,  2

2
2
Did left child,  2

Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
2
Did left child,  2

2
Did left child,  2

Did left child,  2

2
2
Did left child,  2

Did left child,  2

2


In [3]:
for p in annotation_pairs:
  if p.main_annotation.is_valid and p.double_annotation.is_valid:
    maps = {
      "main": p.main_annotation,
      "double": p.double_annotation
    }
    for which_annotation, annotation in maps.items():
      ete_lib.render(annotation, f"binary_parses/{p.identifier}_{which_annotation}.pdf" )